<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Artifical Intelligence - Dr. Mohammad Hossein Rohban
</font>
<br/>
<br/>
Fall 2021

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 5-Q1
            	<br/>
			</font>
    <br/>
    <br/>
<font size=4>
			<br/><br/>
Deadline:  Bahman 17th
                <br/><b>
              Cheating is Strongly Prohibited
                </b><br/><br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

# Personal Data

In [1]:
# Set your student number
student_number = 98105879
Name = 'Ali'
Last_Name = 'Abbasi'

# Rules
- You are not allowed to use provided codes that can be found on the internet. 
- If you want to use a library which is not already imported, you must ask a question on Quera to get the permission of using that.
- Do not hesitate to ask questions on Quera, if you have any.
- This assignment is due Bahman 17th 23:59:59. you can use up to 1 grace day for this assignment and the hard deadline is Bahman 18th 23:59:59.

# Q1 (30Points)

## Mountain Car

The OpenAI Gym library includes a set of Python Reinforcement Learning environments. In this question, we examine the Mountain Car environment of this collection. Mountain Car is a Reinforcement Learning task that aims to learn the policy of climbing a steep hill and reaching the flag-marked goal. Also, the car engine is not powerful enough to climb straight up the hill on the right. Therefore, it must gain enough acceleration by climbing the hill on the left.


In this case, the state of the car is determined by an array containing its position and speed.

| Num | Observation | Min | Max|
| --- | --- | --- | ---|
| 0 | Position | -1.3 | 0.5 |
| 1 | Velocity | -0.07 | 0.07|

The intelligent agent is allowed to perform three movements of right push, no push and left push in each step. The agent's move will be given to the environment and the environment returns the next state along with the movement reward. For each step that the car does not reach the target, a cost of -1 is considered. Use Q-learning to find the optimal policy in each case. To do this, you must complete the functions below.

** Note that you will receive the full score of this question only if you can achieve a score of -115 with your implementation.

<font size=4>
Author: Atoosa Chegini
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

In [2]:
import numpy as np
import random
import gym

In [3]:
env = gym.make("MountainCar-v0")
# initialization (parameters such as learning rate, gamma, number of states, number of actions, and q_table)
learning_rate = 0.5
gamma = 0.9
states_dim, = env.observation_space.shape
n_actions = env.action_space.n

Next cell wants you supplement two functions. First for transforming the continuous space into discrete one (in order to make using q_table feasible), second for updating q_values based on the last action done by agent.

In [4]:
def discretize_state(x, minn, maxn, step = 0.01):
    # return int((x - minn) / step)
    return np.digitize(x, np.arange(minn, maxn, step), right= True)

pos_low, pos_high = -1.2, 0.6
vel_low, vel_high =  -0.07, 0.07
pos_step, vel_step = 0.01, 0.001
n_pos = discretize_state(pos_high, pos_low, pos_high, step=pos_step) + 1
n_vel = discretize_state(vel_high, vel_low, vel_high, step=vel_step) + 1
n_qstates = n_pos * n_vel
q_table = np.zeros((n_qstates, n_actions))

def env_state_to_Q_state(state):
    [position, velocity] = state
    disc_pos = discretize_state(position, pos_low, pos_high, step=pos_step)
    disc_vel = discretize_state(velocity, vel_low, vel_high, step=vel_step)
    return disc_pos * n_vel + disc_vel

def update_q(current_state, next_state, action, reward):
    # best_future_q = np.max(q_table[next_state])
    difference = reward + gamma * np.max(q_table[next_state]) - q_table[current_state, action]
    q_table[current_state, action] += learning_rate * difference

n_qstates

25521

At the following cell, the ends of two functions are getting current action based on the policy and defining the training process respectively.

In [5]:
def get_action(current_state):
    return np.argmax(q_table[current_state], axis=-1)

def epsilon_greedy(current_state, epsilon):
    if epsilon == 0:
        return get_action(current_state)
    return random.choices([get_action(current_state), random.randint(0, 2)], weights=[1-epsilon, epsilon])[0]

def q_learning(episodes = 100000):
    rewards = []
    for i in range(episodes):
        done = False
        state = env_state_to_Q_state(env.reset())
        running_reward = 0
        epsilon_1 = min(1, np.sqrt(2 * i / episodes))
        while not done:
            a = epsilon_greedy(state, 1 - epsilon_1)
            next_state, reward, done, _ = env.step(a)
            next_state = env_state_to_Q_state(next_state)
            update_q(state, next_state, a, reward)
            state = next_state
            running_reward += reward
        rewards.append(running_reward)
        if i % 100 == 0:
            print(f'{i}:\tmean reward = {np.mean(rewards)}\tepsilon = {1 - epsilon_1}')
            rewards = []

def save_policy():
    policy  = np.argmax(q_table, axis=1)
    np.save('policy.npy', policy)

In [13]:
q_learning()
save_policy()

0:	mean reward = -200.0	epsilon = 1.0
100:	mean reward = -200.0	epsilon = 0.9552786404500042
200:	mean reward = -200.0	epsilon = 0.9367544467966324
300:	mean reward = -200.0	epsilon = 0.9225403330758517
400:	mean reward = -200.0	epsilon = 0.9105572809000084
500:	mean reward = -200.0	epsilon = 0.9
600:	mean reward = -200.0	epsilon = 0.8904554884989668
700:	mean reward = -200.0	epsilon = 0.8816784043380077
800:	mean reward = -200.0	epsilon = 0.8735088935932649
900:	mean reward = -200.0	epsilon = 0.8658359213500126
1000:	mean reward = -200.0	epsilon = 0.8585786437626906
1100:	mean reward = -200.0	epsilon = 0.8516760302580868
1200:	mean reward = -200.0	epsilon = 0.8450806661517033
1300:	mean reward = -200.0	epsilon = 0.838754845034029
1400:	mean reward = -200.0	epsilon = 0.8326679946931849
1500:	mean reward = -200.0	epsilon = 0.8267949192431123
1600:	mean reward = -200.0	epsilon = 0.8211145618000169
1700:	mean reward = -200.0	epsilon = 0.8156091108541422
1800:	mean reward = -200.0	epsilon 

In [6]:
# Attention: don't change this function. we will use this to grade your policy which you will hand in with policy.npy
# btw you can use it to see how you are performing. Uncomment two lines which are commented to be able to see what is happening visually.
import time
def score():
    policy, scores = np.load("policy.npy"), []
    for episode in range(1000):
        print(f"******Episode {episode}")
        state, score, done, step = env_state_to_Q_state(env.reset()), 0, False, 0
        while not done:
            # time.sleep(0.04)
            action = policy[state]
            state, reward, done, _ = env.step(action)
            state = env_state_to_Q_state(state)
            step += 1
            score += int(reward)
            # env.render()
        print(f"Score:{score}")
        scores.append(score)
    print(f"Average score over 1000 run : {np.array(scores).mean()}")

score()

******Episode 0
Score:-86
******Episode 1
Score:-87
******Episode 2
Score:-162
******Episode 3
Score:-114
******Episode 4
Score:-105
******Episode 5
Score:-104
******Episode 6
Score:-148
******Episode 7
Score:-114
******Episode 8
Score:-90
******Episode 9
Score:-104
******Episode 10
Score:-87
******Episode 11
Score:-86
******Episode 12
Score:-104
******Episode 13
Score:-103
******Episode 14
Score:-85
******Episode 15
Score:-104
******Episode 16
Score:-103
******Episode 17
Score:-104
******Episode 18
Score:-168
******Episode 19
Score:-97
******Episode 20
Score:-90
******Episode 21
Score:-104
******Episode 22
Score:-86
******Episode 23
Score:-104
******Episode 24
Score:-86
******Episode 25
Score:-104
******Episode 26
Score:-86
******Episode 27
Score:-105
******Episode 28
Score:-104
******Episode 29
Score:-104
******Episode 30
Score:-103
******Episode 31
Score:-88
******Episode 32
Score:-85
******Episode 33
Score:-104
******Episode 34
Score:-104
******Episode 35
Score:-94
******Episode 36

In [7]:
# generating gif from game
import imageio

def generate_gif(env, PATH, t_max=10000):
    with imageio.get_writer(PATH, mode='I') as writer:
        policy = np.load("policy.npy")
        state, score, done, step = env_state_to_Q_state(env.reset()), 0, False, 0
        for i in range(t_max):
            writer.append_data(env.render(mode='rgb_array'))
            action = policy[state]
            state, reward, done, _ = env.step(action)
            state = env_state_to_Q_state(state)
            if done:
              break

generate_gif(env, 'final_performance.gif')

C:\Program Files\Python39\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))
